# Optical Character Recognition (OCR) — Bot-a-thon — Byte Warriors 

### Note: 1. The file locations of the pdf and the json files which contains the table locations should be updated according to your computer before running the code.

### Note: 2. Java 8+ is required to be installed, for running tabula which is used in this code. Other python dependencies are:
pdfplumber

PyPDF2

tabula-py

pandas

operator

re

StringIO

numpy

sys

In [1]:
import pdfplumber
import operator
import re

### Location of the invoice pdf from which bill number, address and table is to be extracted :

In [2]:
invoice = r"H 59248260.pdf"

### Counting the number of pages in the pdf :

In [3]:
from PyPDF2 import PdfFileReader
pdf = PdfFileReader(open(invoice,'rb'))
count = pdf.getNumPages()
print(count)

4


### Extracting the whole text from pdf using pdfplumber (python library) :

In [4]:
text=''
with pdfplumber.open(invoice) as pdf:
    for i in range(0,count):
        page = pdf.pages[i]
        text = text+page.extract_text()
        if i<count-1:
            text = text + "\n"
        else:
            continue

In [5]:
print(text)

Shipper BILL OF LADING B/L No. BQEGMXM016691
Extrade II, S. DE R. L. DE C,V ***COPY NOT NEGOTIABLE*** EXPRESS RELEASE
AV, MANUEL GOMEZ MORIN NO.102 - 15 Shipper's Ref Q0209210109, HQ98
COL. LA ESTACIÓN
RFC EII061116R76 F/Agent's Ref. MXM-CLS-2102-0031
EII061116R76
Consignee (If 'Order' state Notify Party and Address)
Cerveceria Chile S.A
Av Eduardo Frei
Montalva 9600 
8720035 Santiago  
Chile
96547710-1
Notify Party and Address (leave blank if stated above)
Cerveceria Chile S.A
Av Eduardo Frei
Montalva 9600  8720035 
Santiago 226238515 Carl-Zeiss-Strasse 6
Postfach 10 02 54 
55133 Mainz-Hechtsheim
Germany
Pre-Carriage By* Place of Receipt by Pre-Carrier* T  +49 (61 31) 50 30
F  +49 (61 31) 50 32 66
Vessel Port of Loading
Coyhaique 2107E Manzanillo
Port of Discharge Place of Delivery by On-Carrier*
Valparaiso
Carrier's Receipt Particulars furnished by shipper - Carrier not responsible
Marks and Nos Container No. Number and kind of packages; description of goods Gross Weight Measurement


### Checking for the Bill Number type and printing it :

In [6]:
temp_bill = ''
for row in text.split('\n'):
    if "Carl-Zeiss-Strasse" in row:
        temp_bill = "Eagle"
        break
    if "CMA" in row and "CGM" in row:
        temp_bill = "CMA"
        break
    if "jurisdiction" in row and "clause" in row:
        temp_bill = "Maersk"
        break
print(temp_bill)

Eagle


In [7]:
bill="Bill Number: "
flag = 0
if temp_bill == "CMA":
    billno = re.search("[A-Z]{3}\d{7}",text)
elif temp_bill == "Maersk":
    billno = re.search("\d{9}",text)
elif temp_bill == "Eagle":
    billno = re.search("[A-Z]{7}\d{6}",text)
bill = bill + billno.group() + "\n"
print(bill)

Bill Number: BQEGMXM016691



### Finding the "Shipper Address" using conditional statements and keywords :

In [8]:
flag = 0
det = "Shipper Address:" + "\n"
for row in text.split('\n'):
    if "Shipper" in row or "SHIPPER" in row:
        flag = 4
    elif  flag != 0:
        if "Export references Svc Contract" in row:
            continue
        det = det + row
        det = det + "\n"
        print(row)
        flag = flag - 1
        if flag == 0:
            break

Extrade II, S. DE R. L. DE C,V ***COPY NOT NEGOTIABLE*** EXPRESS RELEASE
COL. LA ESTACIÓN
RFC EII061116R76 F/Agent's Ref. MXM-CLS-2102-0031
EII061116R76
Consignee (If 'Order' state Notify Party and Address)


### Removing the common unwanted information through replace and uncommon items through Regex :

In [9]:
det = det.replace("VOYAGE NUMBER",'')
det = det.replace(" SHIPPER",'SHIPPER ADDRESS:')
det = det.replace(" SHIPPER ADDRESS:",'')
det = det.replace(" WAYBILL NUMBER",'')
det = det.replace(" WAYBILL",'')
det = det.replace("NON NEGOTIABLE",'')
det = det.replace(" ***COPY NOT NEGOTIABLE*** EXPRESS RELEASE",'')
det = det.replace("Export references",'')
det = det.replace("Svc Contract",'')
det = det.replace(" F/Agent's Ref.",'')
det = det.replace("Consignee (If 'Order' state Notify Party and Address)",'')
det = det.replace(" \n",'')
det = re.sub(r'\s{1}\d{1}[A-Z]{2}\d{1}[A-Z]{2}\d{1}[A-Z]{2}', '', det)
det = re.sub(r'\s{1}[A-Z]{3}-{1}[A-Z]{3}-{1}\d{4}-{1}\d{4}', '', det)
det = re.sub(r'\s{1}\d{9}', '', det)
print(det)

Shipper Address:
Extrade II, S. DE R. L. DE C,V
COL. LA ESTACIÓN
RFC EII061116R76
EII061116R76




### Adding Shipper address to the list :

In [10]:
new_det = det
new_det = new_det + "\n"
new_det = new_det.split('\n')
bill = [bill]
list1 = []
list1 = list1 + bill
list1 = list1 + new_det

In [11]:
mark = 0
for row in text.split('\n'):
    if "This contract is subject to the terms, conditions and exceptions, including the law & jurisdiction clause" in row:
        mark = 2
        print(mark)

In [12]:
mark2 = 0
for row in text.split('\n'):
    if "NON-NEGOTIABLE" in row:
        mark2 = 1

### Finding the "Consignee Address" using conditional statements and keywords :

In [13]:
flag = 0
count2 = 0
con = "Consignee Address:" + "\n"
for row in text.split('\n'):
    if "Consignee" in row or "CONSIGNEE" in row:
           if mark2 == 1:
             flag = 6
           else:
             flag = 4
    elif  flag != 0:
        if "Notify Party (see clause 22) the Carrier reasonable notice in writing." in row:
            break
        con = con + row
        con = con + "\n"
        print(row)
        flag = flag - 1
        if flag == 0:
            break

Cerveceria Chile S.A
Av Eduardo Frei
Montalva 9600 
8720035 Santiago  


In [14]:
con = con.replace("This contract is subject to the terms, conditions and exceptions, including the law & jurisdiction clause",'')
con = con.replace("and limitation of liability & declared value clauses, of the current Maersk Bill of Lading (available from",'')
con = con.replace("\n\n","\n")
con = con.replace(" the Carrier, its agents and at terms.maersk.com/carriage), which are applicable with logical","")
con = con.replace(" sued under this contract, the Shipper on entering into this contract does so on his own behalf and as","")
con = con.replace("Lizi.Dvori@hacarem.com its agents and at https://terms.sealandmaersk.com/europe/carriage), which are applicable with logical\n",'')
con = con.replace("This contract is subject to the terms and conditions, including the law & jurisdiction clause and limitation\n",'')
con = con.replace(" of liability & declared value clauses,  of the current Sealand Bill of Lading (available from the carrier,",'')
con = re.sub(r'\s{1}[A-Z]{1}\d{10}', '', con)
print(con)

Consignee Address:
Cerveceria Chile S.A
Av Eduardo Frei
Montalva 9600 
8720035 Santiago  



In [15]:
new_con = con
new_con = new_con.split('\n')
print(new_con)
list1 = list1 + new_con

['Consignee Address:', 'Cerveceria Chile S.A', 'Av Eduardo Frei', 'Montalva 9600 ', '8720035 Santiago  ', '']


### Finding the "Notify Party Address" using conditional statements and keywords :

In [16]:
flag = 0
con = ""
bounce = 4
notify =  "Notify Party Address:" + "\n"
numoftimes = 0
for row in text.split('\n'):
    if "Notify Party" in row or "NOTIFY PARTY" in row or "Notify" in row :
        numoftimes = numoftimes +1
if numoftimes ==1:
    for row in text.split('\n'):
        if "Notify Party" in row or "NOTIFY PARTY" in row or "Notify" in row :
               flag = 4
               if "reasonable notice in writing" in row:
                    flag = 7
        elif  flag != 0:
            notify = notify + row
            notify = notify + "\n"
            print(row)
            flag = flag - 1
            if flag == 0:
                break
elif numoftimes ==2:
    for row in text.split('\n'):
        if "Notify Party" in row or "NOTIFY PARTY" in row or "Notify" in row :
               flag = flag +1
        if flag == 2:
            flag=3
            continue
        if  flag == 3:
            print(row)
            notify = notify + row
            notify = notify + "\n"
            bounce = bounce - 1
            if bounce == 0:
                break
        else:
             continue

Cerveceria Chile S.A
Av Eduardo Frei
Montalva 9600  8720035 
Santiago 226238515 Carl-Zeiss-Strasse 6


In [17]:
notify = notify.replace('Delivery will be made to the Consignee or his authorised agent on production of reasonable proof of','')
notify = notify.replace("identity (and, in the case of an agent, reasonable proof of authority) without production of this waybill.",'')
notify = notify.replace("The Carrier shall be under no liability whatsoever for misdelivery unless caused by the Carrier's\n",'')
notify = notify.replace("negligence.\n",'')
notify = notify.replace(" CARRIER: CMA CGM Société Anonyme au Capital de 234 988 330 Euros",'')
notify = notify.replace(" Head Office: 4, quai d'Arenc - 13002 Marseille - France",'')
notify = notify.replace("           Tel: (33) 4 88 91 90 00 -   Fax: (33) 4 88 91 90 95",'')
notify = notify.replace(" Onward inland routing (Not part of Carriage as defined in clause 1. For account and risk of Merchant)",'')
notify = notify.replace(" Carl-Zeiss-Strasse 6",'')
print(notify)

Notify Party Address:
Cerveceria Chile S.A
Av Eduardo Frei
Montalva 9600  8720035 
Santiago 226238515



In [18]:
flag = 0
matches = []
for row in text.split('\n'):
    if "Notify Party" in row or "NOTIFY PARTY" in row or "Notify" in row :
        matches.append(row)
print(matches)

["Consignee (If 'Order' state Notify Party and Address)", 'Notify Party and Address (leave blank if stated above)']


In [19]:
new_notify = notify
new_notify = new_notify.split('\n')
list1 = list1 + new_notify

### A list which contains the Bill Number and all the 3 addresses :

In [20]:
list1

['Bill Number: BQEGMXM016691\n',
 'Shipper Address:',
 'Extrade II, S. DE R. L. DE C,V',
 'COL. LA ESTACIÓN',
 'RFC EII061116R76',
 'EII061116R76',
 '',
 '',
 '',
 'Consignee Address:',
 'Cerveceria Chile S.A',
 'Av Eduardo Frei',
 'Montalva 9600 ',
 '8720035 Santiago  ',
 '',
 'Notify Party Address:',
 'Cerveceria Chile S.A',
 'Av Eduardo Frei',
 'Montalva 9600  8720035 ',
 'Santiago 226238515',
 '']

## Table Extraction :

In [21]:
import pdfplumber
import pandas as pd
import pdfplumber
import tabula

### Identifying the type of the table:

In [22]:
temp = ''
for row in text.split('\n'):
    if "Carl-Zeiss-Strasse" in row:
        from PyPDF2 import PdfFileReader
        pdf = PdfFileReader(open(invoice,'rb'))
        count = pdf.getNumPages()
        if count == 3:
            temp = 'Eagle1'
        if count == 4 :
            temp = 'Eagle2'
    if "CMA" in row and "CGM" in row:
        temp = 'CMA'
    if "jurisdiction" in row and "clause" in row:
        from PyPDF2 import PdfFileReader
        pdf = PdfFileReader(open(invoice,'rb'))
        count = pdf.getNumPages()
        if count == 1:
            temp = 'Maersk1'
        if count == 2 :
            temp = 'Maersk2'
print(temp)

Eagle2


### Extracting the table data using Tabula python module, using the saved table loction files, present in json format & according to the type of the table :

In [23]:
if temp == 'CMA':
    df = tabula.io.read_pdf_with_template(invoice,r"C:\Users\srisu\Downloads\TableLocations\Normal pdf table locations.json",guess = False, multiple_tables = True, stream=True)
    print(df)
if temp == 'Eagle1':
    df = tabula.io.read_pdf_with_template(invoice,r"C:\Users\srisu\Downloads\TableLocations\Blue eagle single table locations.json",guess = False, multiple_tables = True, stream=True)
    print(df)
if temp == 'Eagle2':
    df = tabula.io.read_pdf_with_template(invoice,r"C:\Users\srisu\Downloads\TableLocations\Blue eagle 2 table locations.json",guess = False, multiple_tables = True, stream=True)
    print(df)
if temp == 'Maersk1':
    df = tabula.io.read_pdf_with_template(invoice,r"C:\Users\srisu\Downloads\TableLocations\Maersk1 table locations.json",guess = False, multiple_tables = False, stream=True)
    print(df)
if temp == 'Maersk2':
    df = tabula.read_pdf_with_template(invoice,r"C:\Users\srisu\Downloads\TableLocations\Maersk2 table locations.json",guess = False, multiple_tables = True, stream=True)
    print(df)

[                                    Carrier's Receipt  \
0                         Marks and Nos Container No.   
1                                         CAAU5464770   
2                                   Seal  MXJFH003865   
3                                       4508757839_16   
4                                          0015545700   
5   FFAU1335486 Seal  MXJFH0038814508757839_16 184...   
6                                                 NaN   
7                                         HLBU3242640   
8                                   Seal  MXJFH003868   
9                                       4508757839_16   
10                                         0015545701   
11                                                NaN   
12                                        HLBU3378534   
13                                  Seal  MXJFH003883   
14                                      4508757839_16   
15                                         0015545697   
16                            

### Checking the number of tables :

In [24]:
length = len(df)
print(length)

2


In [25]:
df

[                                    Carrier's Receipt  \
 0                         Marks and Nos Container No.   
 1                                         CAAU5464770   
 2                                   Seal  MXJFH003865   
 3                                       4508757839_16   
 4                                          0015545700   
 5   FFAU1335486 Seal  MXJFH0038814508757839_16 184...   
 6                                                 NaN   
 7                                         HLBU3242640   
 8                                   Seal  MXJFH003868   
 9                                       4508757839_16   
 10                                         0015545701   
 11                                                NaN   
 12                                        HLBU3378534   
 13                                  Seal  MXJFH003883   
 14                                      4508757839_16   
 15                                         0015545697   
 16           

### Converting the table columns list to Data Frame, then to string :

In [26]:
import pandas as pd
df = [df]
listreal8 = pd.DataFrame(df)
pq=listreal8.to_string()
print(pq)

### Assigning the columns of the table, one by one using keywords and conditional statements :

In [27]:
str9 = pq
if temp == 'CMA':
    string123 = "MARKS "
elif temp =="Maersk1" or temp =="Maersk2":
    string123 = ""
else:
    string123 = ""
line_items = []
for line in str9.split('\n'):
    try:
        s_no, *container1 = line.split()
        if "MARKS" in line.split():
            string123 = ""
        container1 = ' '.join(container1)
        print(container1)
        string123 = string123 + container1
        string123 = string123 + "\n"
    except ValueError:
        break

1
Carrier's Receipt \
Marks and Nos Container No.
CAAU5464770
Seal MXJFH003865
4508757839_16
0015545700
FFAU1335486 Seal MXJFH0038814508757839_16 1848 Case(s) COR EXT 6 PACAK 24/330 CHILE 80620 ; 0015545696 D
NaN
HLBU3242640
Seal MXJFH003868
4508757839_16
0015545701
NaN
HLBU3378534
Seal MXJFH003883
4508757839_16
0015545697
NaN


In [28]:
string123 = string123.replace("NaN",'_')
string123 = string123.replace(" CERTIFIED","")
string123 = string123.replace(" CERTIFI","")
string123 = string123.replace(" E COPY","")
string123 = string123.replace(" COPY","")
string123 = string123.replace(" TRUE","")
print(string123)

1
Carrier's Receipt \
Marks and Nos Container No.
CAAU5464770
Seal MXJFH003865
4508757839_16
0015545700
FFAU1335486 Seal MXJFH0038814508757839_16 1848 Case(s) COR EXT 6 PACAK 24/330 CHILE 80620 ; 0015545696 D
_
HLBU3242640
Seal MXJFH003868
4508757839_16
0015545701
_
HLBU3378534
Seal MXJFH003883
4508757839_16
0015545697
_



In [29]:
if temp == "CMA" or temp =="Eagle1" or temp == "Eagle2":
    call = 0
    if temp == 'CMA':
        string12 = "No. "
    elif temp =="Maersk1" or temp =="Maersk2":
        string12 = "Kind "
    else:
        string12 = ""
    for line in str9.split('\n'):
        try:
            if "KIND" in line.split() or "kind" in line.split() or "Kind" in line.split():
                call = call + 1
            if call == 1:
                s_no, *package1 = line.split()
                package1 = ' '.join(package1)
                print(package1)
                string12 = string12 + package1
                string12 = string12 + "\n"
        except ValueError:
            break

Number and kind of packages; description of goods
1 X 40' High Cube Dry SLAC
NaN
1842 Case(s) COR EXT 6 PACK 24/330 CHILE 80620 ;
Customs Code 22030001
1 X R40' High Cube Dry SLAC Customs Code 22030001 Total: 1842 packages.FT
Total: 1848 packages.
1 X 40' High Cube Dry SLAC
NaN
1848 Case(s) COR EXT 6 PACK 24/330 CHILE 80620 ;
Customs Code 22030001
Total: 1848 packages.
1 X 40' High Cube Dry SLAC
NaN
1848 Case(s) COR EXT 6 PACK 24/330 CHILE 80620 ;
Customs Code 22030001
said to contain, received for shipment


In [30]:
if temp == "CMA" or temp =="Eagle1" or temp == "Eagle2":
    string12 = string12.replace("NaN",'_')
    print(string12)

Number and kind of packages; description of goods
1 X 40' High Cube Dry SLAC
_
1842 Case(s) COR EXT 6 PACK 24/330 CHILE 80620 ;
Customs Code 22030001
1 X R40' High Cube Dry SLAC Customs Code 22030001 Total: 1842 packages.FT
Total: 1848 packages.
1 X 40' High Cube Dry SLAC
_
1848 Case(s) COR EXT 6 PACK 24/330 CHILE 80620 ;
Customs Code 22030001
Total: 1848 packages.
1 X 40' High Cube Dry SLAC
_
1848 Case(s) COR EXT 6 PACK 24/330 CHILE 80620 ;
Customs Code 22030001
said to contain, received for shipment



In [31]:
call2 = 0
string1 ="GROSS "
for line in str9.split('\n'):
    try:
        if "Measurement" in line.split() or "MEASUREMENT" in line.split():
            call2 = call2 + 1
        if call2 == 1:
            if "MARKS" in line.split() or "Nos" in line.split():
                break
            s_no, *weight1 = line.split()
            weight1 = ' '.join(weight1)
            print(weight1)
            string1 = string1 + weight1
            string1 = string1 + "\n"
    except ValueError:
        break

Gross Weight Measurement
26.064,30 KGM 44,210 CBM
NaN NaN
NaN NaN
NaN NaN
26.149,20 KGM 44,350 CBM
NaN NaN
26.149,20 KGM 44,350 CBM
NaN NaN
NaN NaN
NaN NaN
NaN NaN
26.149,20 KGM 44,350 CBM
NaN NaN
NaN NaN
NaN NaN


In [32]:
string1 = string1.replace("NaN",'_')
print(string1)

GROSS Gross Weight Measurement
26.064,30 KGM 44,210 CBM
_ _
_ _
_ _
26.149,20 KGM 44,350 CBM
_ _
26.149,20 KGM 44,350 CBM
_ _
_ _
_ _
_ _
26.149,20 KGM 44,350 CBM
_ _
_ _
_ _



### Checking for a second table and assigning columns according to their types :

In [33]:
if temp == "CMA" or temp == "Eagle2" or temp =="Maersk2":    
    call = 0
    if temp == "CMA" or temp == "Eagle2":
        newstring123 = "\n"  + "MARKS "
    else:
        newstring123 = "\n"
    for line in str9.split('\n'):
        try:
            if "NOS" in line.split() or "Nos" in line.split():
                call = call + 1
                if "MARKS" in line.split():
                    newstring123 = ""
            if call == 2:
                s_no, *container2 = line.split()
                container2 = ' '.join(container2)
                print(container2)
                newstring123 = newstring123 + container2
                newstring123 = newstring123 + "\n"
        except ValueError:
            break 

NaN NaN Marks and Nos Container No \
NaN
TCNU1815298
Seal MXJFH003869
4508757839_16
0015545695
NaN
TRHU4943845
Seal MXJFH003875
4508757839_16
0015545699
NaN
TRHU4979909
Seal MXJFH003858
4508757839_16
0015545698
Bkg: 59248260
NaN


In [34]:
if temp == "CMA" or temp == "Eagle2" or temp =="Maersk2":
    newstring123 =newstring123.replace("NaN",'_')
    print(newstring123)


MARKS _ _ Marks and Nos Container No \
_
TCNU1815298
Seal MXJFH003869
4508757839_16
0015545695
_
TRHU4943845
Seal MXJFH003875
4508757839_16
0015545699
_
TRHU4979909
Seal MXJFH003858
4508757839_16
0015545698
Bkg: 59248260
_



In [35]:
if temp == "CMA" or temp == "Eagle2":
        call = 0
        if temp == "CMA" or temp == "Eagle2":
            newstring12 = "\n" + "NO. "
        else:
            newstring12 = "\n"
        for line in str9.split('\n'):
            try:
                if "DESCRIPTION" in line.split() or "kind" in line.split():
                    call = call + 1
                if call == 2:
                    s_no, *package2 = line.split()
                    package2 = ' '.join(package2)
                    print(package2)
                    newstring12 = newstring12 + package2
                    newstring12 = newstring12 + "\n"
            except ValueError:
                break

and kind of Packages; description of goods Gross Weight \
Total: 1848 packages. NaN
1 X 40' High Cube Dry SLAC 26.149,20 KGM
NaN NaN
1848 Case(s) COR EXT 6 PACK 24/330 CHILE 80620 ; NaN
Customs Code 22030001 NaN
Total: 1848 packages. NaN
1 X 40' High Cube Dry SLAC 26.149,20 KGM
NaN NaN
1848 Case(s) COR EXT 6 PACK 24/330 CHILE 80620 ; NaN
Customs Code 22030001 NaN
Total: 1848 packages. NaN
1 X 40' High Cube Dry SLAC 26.149,20 KGM
NaN NaN
1848 Case(s) COR EXT 6 PACK 24/330 CHILE 80620 ; NaN
Customs Code 22030001 NaN
NaN NaN
Total: 1848 packages. NaN


In [36]:
if temp == "CMA" or temp == "Eagle2":
      newstring12 =newstring12.replace("NaN",'_')
      print(newstring12)


NO. and kind of Packages; description of goods Gross Weight \
Total: 1848 packages. _
1 X 40' High Cube Dry SLAC 26.149,20 KGM
_ _
1848 Case(s) COR EXT 6 PACK 24/330 CHILE 80620 ; _
Customs Code 22030001 _
Total: 1848 packages. _
1 X 40' High Cube Dry SLAC 26.149,20 KGM
_ _
1848 Case(s) COR EXT 6 PACK 24/330 CHILE 80620 ; _
Customs Code 22030001 _
Total: 1848 packages. _
1 X 40' High Cube Dry SLAC 26.149,20 KGM
_ _
1848 Case(s) COR EXT 6 PACK 24/330 CHILE 80620 ; _
Customs Code 22030001 _
_ _
Total: 1848 packages. _



In [37]:
if temp == "CMA" or temp == "Eagle2" or temp == "Maersk2":    
        call = 0
        if temp == "CMA":
            newstring1 = "\n" + "GROSS "
        else:
            newstring1 = "\n"
        for line in str9.split('\n'):
            try:
                if "MEASUREMENT" in line.split() or "Measurement" in line.split():
                    call = call + 1
                if call == 2:
                    s_no, *weight2 = line.split()
                    weight2 = ' '.join(weight2)
                    print(weight2)
                    newstring1 = newstring1 + weight2
                    newstring1 = newstring1 + "\n"
            except ValueError:
                break


NaN
44,350 CBM
NaN
NaN
NaN
NaN
44,350 CBM
NaN
NaN
NaN
NaN
44,350 CBM
NaN
NaN
NaN
NaN
NaN


### Have replaced the "NaN" values with "_" instead of empty character because pandas DataFrame will concatenate  the cells, if the cells are empty.

In [38]:
if temp == "CMA" or temp == "Eagle2" or temp == "Maersk2":
    newstring1 =newstring1.replace("NaN",'_')
    print(newstring1)



_
44,350 CBM
_
_
_
_
44,350 CBM
_
_
_
_
44,350 CBM
_
_
_
_
_



In [39]:
if temp == "CMA" or temp == "Eagle2": 
    string123 = string123+ "\n" + "\n" + "TABLE 2:"
    string12 = string12+ "\n" + "\n"
    string1 = string1+ "\n" + "\n"

In [40]:
if temp == "Maersk2":
    if length>1:
        if temp == "CMA" or temp == "Eagle2": 
            string123 = string123+ "\n" + "\n" + "TABLE 2:"
            string12 = string12+ "\n" + "\n"
            string1 = string1+ "\n" + "\n"

In [41]:
if temp == "CMA" or temp == "Eagle2":
    print(newstring1)



_
44,350 CBM
_
_
_
_
44,350 CBM
_
_
_
_
44,350 CBM
_
_
_
_
_



### Combining the columns of the first table into a data frame:

In [42]:
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

import pandas as pd
TESTDATA = StringIO(string123)
f = pd.read_csv(TESTDATA, sep="\n")

In [43]:
if temp != "Maersk2" and temp!= "Maersk1":
    TESTDATA = StringIO(string12)
    g = pd.read_csv(TESTDATA, sep="\n")

In [44]:
TESTDATA = StringIO(string1)
h = pd.read_csv(TESTDATA, sep="\n")

### Combining Table 2 columns into a Data Frame if present :

In [45]:
if temp == "Maersk2":
    if length ==1:
        pass
    elif length>1:
        TESTDATA = newstring123.split("\n")
        i = pd.DataFrame(TESTDATA)
if temp == "CMA" or temp == "Eagle2":
    TESTDATA = StringIO(newstring123)
    i = pd.read_csv(TESTDATA, sep="\n")

In [46]:
if temp == "CMA" or temp == "Eagle2":
    TESTDATA = StringIO(newstring12)
    j = pd.read_csv(TESTDATA, sep="\n")

In [47]:
if temp == "Maersk2":
    if length ==1:
        pass
    elif length>1:
        TESTDATA = StringIO(newstring1)
        k = pd.read_csv(TESTDATA, sep="\n")
if temp == "CMA" or temp == "Eagle2":
    TESTDATA = StringIO(newstring1)
    k = pd.read_csv(TESTDATA, sep="\n")

In [48]:
if temp == "CMA" or temp == "Eagle2" or temp == "Eagle1":
    frames1 = [f,g,h]
    result1 = pd.concat(frames1,keys=["a","b","c"],axis = 1)
    
elif temp =="Maersk1" or temp == "Maersk2":
    frames4 = [f,h]
    result1 = pd.concat(frames4,keys=["a","b"],axis = 1)

In [49]:
if temp == "CMA" or temp == "Eagle2":
        frames2 = [i,j,k]
        newresult = pd.concat(frames2,keys=["a","b","c"],axis = 1)
elif temp =="Maersk2":
    if length >1:
        frames3 = [i,k]
        newresult = pd.concat(frames3,keys=["a","b"],axis = 1)

In [50]:
result1

,a,b,c
,1,Number and kind of packages; description of goods,GROSS Gross Weight Measurement
0,Carrier's Receipt \,1 X 40' High Cube Dry SLAC,"26.064,30 KGM 44,210 CBM"
1,Marks and Nos Container No.,_,_ _
2,CAAU5464770,1842 Case(s) COR EXT 6 PACK 24/330 CHILE 80620 ;,_ _
3,Seal MXJFH003865,Customs Code 22030001,_ _
4,4508757839_16,1 X R40' High Cube Dry SLAC Customs Code 22030...,"26.149,20 KGM 44,350 CBM"
5,0015545700,Total: 1848 packages.,_ _
6,FFAU1335486 Seal MXJFH0038814508757839_16 1848...,1 X 40' High Cube Dry SLAC,"26.149,20 KGM 44,350 CBM"
7,_,_,_ _
8,HLBU3242640,1848 Case(s) COR EXT 6 PACK 24/330 CHILE 80620 ;,_ _


In [51]:
if temp == "CMA":
    result1.columns = ['MARKS AND NOS, CONTAINER AND SEALS', 'NO AND KINDOF PACKAGES & DESCRIPTION OF PACKAGES AND GOODS AS STATED BY SHIPPER', 'GROSS WEIGHT OF CARGO, TARE & MEASUREMENT']
if temp == "Eagle2" or temp == "Eagle1":
    result1.columns = ['MARKS AND NOS, CONTAINER AND SEALS', 'NO AND KINDOF PACKAGES & DESCRIPTION OF PACKAGES AND GOODS AS STATED BY SHIPPER, GROSS WEIGHT OF CARGO', 'TARE & MEASUREMENT']

In [52]:
if temp == "CMA":
    newresult.columns = ['MARKS AND NOS, CONTAINER AND SEALS', 'NO AND KINDOF PACKAGES & DESCRIPTION OF PACKAGES AND GOODS AS STATED BY SHIPPER', 'GROSS WEIGHT OF CARGO, TARE & MEASUREMENT']
if temp == "Eagle2":
    newresult.columns = ['MARKS AND NOS, CONTAINER AND SEALS', 'NO AND KINDOF PACKAGES & DESCRIPTION OF PACKAGES AND GOODS AS STATED BY SHIPPER, GROSS WEIGHT OF CARGO', 'TARE & MEASUREMENT']

In [53]:
if temp == "Maersk1" or temp == "Maersk2":
    result1.columns = ['Kind of Packages; Description of goods; Marks and Numbers; Container No./Seal No.; Weight','Measurement']

In [54]:
if temp == "Maersk2":
    if length > 1:
        newresult.columns = ['Kind of Packages; Description of goods; Marks and Numbers; Container No./Seal No.; Weight','Measurement']

## Table 1:

In [55]:
result1

,"MARKS AND NOS, CONTAINER AND SEALS","NO AND KINDOF PACKAGES & DESCRIPTION OF PACKAGES AND GOODS AS STATED BY SHIPPER, GROSS WEIGHT OF CARGO",TARE & MEASUREMENT
0,Carrier's Receipt \,1 X 40' High Cube Dry SLAC,"26.064,30 KGM 44,210 CBM"
1,Marks and Nos Container No.,_,_ _
2,CAAU5464770,1842 Case(s) COR EXT 6 PACK 24/330 CHILE 80620 ;,_ _
3,Seal MXJFH003865,Customs Code 22030001,_ _
4,4508757839_16,1 X R40' High Cube Dry SLAC Customs Code 22030...,"26.149,20 KGM 44,350 CBM"
5,0015545700,Total: 1848 packages.,_ _
6,FFAU1335486 Seal MXJFH0038814508757839_16 1848...,1 X 40' High Cube Dry SLAC,"26.149,20 KGM 44,350 CBM"
7,_,_,_ _
8,HLBU3242640,1848 Case(s) COR EXT 6 PACK 24/330 CHILE 80620 ;,_ _
9,Seal MXJFH003868,Customs Code 22030001,_ _


## Table 2 (if present):

In [56]:
if length >1:
    newresult

In [57]:
import numpy as np
if temp == "CMA" or temp == "Eagle2":      
    newresult = newresult.replace(np.nan, '')
    newresult

In [58]:
if temp == "CMA" or temp == "Eagle2":    
    final = result1.append(newresult, ignore_index=True)
else:
    final = result1

In [59]:
final = final.replace(np.nan, '')
final = pd.DataFrame(final)
final

,"MARKS AND NOS, CONTAINER AND SEALS","NO AND KINDOF PACKAGES & DESCRIPTION OF PACKAGES AND GOODS AS STATED BY SHIPPER, GROSS WEIGHT OF CARGO",TARE & MEASUREMENT
0,Carrier's Receipt \,1 X 40' High Cube Dry SLAC,"26.064,30 KGM 44,210 CBM"
1,Marks and Nos Container No.,_,_ _
2,CAAU5464770,1842 Case(s) COR EXT 6 PACK 24/330 CHILE 80620 ;,_ _
3,Seal MXJFH003865,Customs Code 22030001,_ _
4,4508757839_16,1 X R40' High Cube Dry SLAC Customs Code 22030...,"26.149,20 KGM 44,350 CBM"
5,0015545700,Total: 1848 packages.,_ _
6,FFAU1335486 Seal MXJFH0038814508757839_16 1848...,1 X 40' High Cube Dry SLAC,"26.149,20 KGM 44,350 CBM"
7,_,_,_ _
8,HLBU3242640,1848 Case(s) COR EXT 6 PACK 24/330 CHILE 80620 ;,_ _
9,Seal MXJFH003868,Customs Code 22030001,_ _


## Final table (table1 + table2):

In [60]:
final

,"MARKS AND NOS, CONTAINER AND SEALS","NO AND KINDOF PACKAGES & DESCRIPTION OF PACKAGES AND GOODS AS STATED BY SHIPPER, GROSS WEIGHT OF CARGO",TARE & MEASUREMENT
0,Carrier's Receipt \,1 X 40' High Cube Dry SLAC,"26.064,30 KGM 44,210 CBM"
1,Marks and Nos Container No.,_,_ _
2,CAAU5464770,1842 Case(s) COR EXT 6 PACK 24/330 CHILE 80620 ;,_ _
3,Seal MXJFH003865,Customs Code 22030001,_ _
4,4508757839_16,1 X R40' High Cube Dry SLAC Customs Code 22030...,"26.149,20 KGM 44,350 CBM"
5,0015545700,Total: 1848 packages.,_ _
6,FFAU1335486 Seal MXJFH0038814508757839_16 1848...,1 X 40' High Cube Dry SLAC,"26.149,20 KGM 44,350 CBM"
7,_,_,_ _
8,HLBU3242640,1848 Case(s) COR EXT 6 PACK 24/330 CHILE 80620 ;,_ _
9,Seal MXJFH003868,Customs Code 22030001,_ _


In [61]:
str1 = ["\nInvoice Table:\n"]
list1 = list1 + str1
listToStr = '\n'.join(map(str, list1))
listToStr = listToStr.replace("None\n",'')
listToStr = listToStr.replace("None",'')
print(listToStr)

Bill Number: BQEGMXM016691

Shipper Address:
Extrade II, S. DE R. L. DE C,V
COL. LA ESTACIÓN
RFC EII061116R76
EII061116R76



Consignee Address:
Cerveceria Chile S.A
Av Eduardo Frei
Montalva 9600 
8720035 Santiago  

Notify Party Address:
Cerveceria Chile S.A
Av Eduardo Frei
Montalva 9600  8720035 
Santiago 226238515


Invoice Table:



In [62]:
strq = []
strq = listToStr.split('\n')

In [63]:
strq

['Bill Number: BQEGMXM016691',
 '',
 'Shipper Address:',
 'Extrade II, S. DE R. L. DE C,V',
 'COL. LA ESTACIÓN',
 'RFC EII061116R76',
 'EII061116R76',
 '',
 '',
 '',
 'Consignee Address:',
 'Cerveceria Chile S.A',
 'Av Eduardo Frei',
 'Montalva 9600 ',
 '8720035 Santiago  ',
 '',
 'Notify Party Address:',
 'Cerveceria Chile S.A',
 'Av Eduardo Frei',
 'Montalva 9600  8720035 ',
 'Santiago 226238515',
 '',
 '',
 'Invoice Table:',
 '']

In [64]:
df = pd.DataFrame(strq)

In [65]:
df.columns=['Bill No & Address']

### Data Frame of Bill Number and Address:

In [66]:
df

,Bill No & Address
0,Bill Number: BQEGMXM016691
1,
2,Shipper Address:
3,"Extrade II, S. DE R. L. DE C,V"
4,COL. LA ESTACIÓN
5,RFC EII061116R76
6,EII061116R76
7,
8,
9,


In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 1 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Bill No & Address  25 non-null     object
dtypes: object(1)
memory usage: 328.0+ bytes


In [68]:
frames5 = [df,final]
final_result = df.append(final, ignore_index=True)

In [69]:
final_result

,Bill No & Address,"MARKS AND NOS, CONTAINER AND SEALS","NO AND KINDOF PACKAGES & DESCRIPTION OF PACKAGES AND GOODS AS STATED BY SHIPPER, GROSS WEIGHT OF CARGO",TARE & MEASUREMENT
0,Bill Number: BQEGMXM016691,NaN,NaN,NaN
1,,NaN,NaN,NaN
2,Shipper Address:,NaN,NaN,NaN
3,"Extrade II, S. DE R. L. DE C,V",NaN,NaN,NaN
4,COL. LA ESTACIÓN,NaN,NaN,NaN
...,...,...,...,...
56,NaN,Seal MXJFH003858,_ _,_
57,NaN,4508757839_16,1848 Case(s) COR EXT 6 PACK 24/330 CHILE 80620...,_
58,NaN,0015545698,Customs Code 22030001 _,_
59,NaN,Bkg: 59248260,_ _,_


In [70]:
if temp == "CMA":
    final_result_new = final_result.reindex(['Bill No & Address','MARKS AND NOS, CONTAINER AND SEALS', 'NO AND KINDOF PACKAGES & DESCRIPTION OF PACKAGES AND GOODS AS STATED BY SHIPPER', 'GROSS WEIGHT OF CARGO, TARE & MEASUREMENT'],axis='columns')
if temp == "Eagle1" or temp == "Eagle2":
    final_result_new = final_result.reindex(['Bill No & Address','MARKS AND NOS, CONTAINER AND SEALS', 'NO AND KINDOF PACKAGES & DESCRIPTION OF PACKAGES AND GOODS AS STATED BY SHIPPER, GROSS WEIGHT OF CARGO', 'TARE & MEASUREMENT'],axis='columns')

In [71]:
if temp == "Maersk1" or temp == "Maersk2":
    final_result_new = final_result.reindex(['Bill No & Address','Kind of Packages; Description of goods; Marks and Numbers; Container No./Seal No.; Weight','Measurement'],axis='columns')

## Final Data Frame with Bill Number, Address & Table data :

In [72]:
final_result_new = final_result_new.replace(np.nan, '')
final_result_new

,Bill No & Address,"MARKS AND NOS, CONTAINER AND SEALS","NO AND KINDOF PACKAGES & DESCRIPTION OF PACKAGES AND GOODS AS STATED BY SHIPPER, GROSS WEIGHT OF CARGO",TARE & MEASUREMENT
0,Bill Number: BQEGMXM016691,,,
1,,,,
2,Shipper Address:,,,
3,"Extrade II, S. DE R. L. DE C,V",,,
4,COL. LA ESTACIÓN,,,
...,...,...,...,...
56,,Seal MXJFH003858,_ _,_
57,,4508757839_16,1848 Case(s) COR EXT 6 PACK 24/330 CHILE 80620...,_
58,,0015545698,Customs Code 22030001 _,_
59,,Bkg: 59248260,_ _,_


In [73]:
final_result_new.isnull().any()

Bill No & Address                                                                                         False
MARKS AND NOS, CONTAINER AND SEALS                                                                        False
NO AND KINDOF PACKAGES & DESCRIPTION OF PACKAGES AND GOODS AS STATED BY SHIPPER, GROSS WEIGHT OF CARGO    False
TARE & MEASUREMENT                                                                                        False
dtype: bool

## Creating an Excel Document with the data frame :

In [74]:
final_result_new.to_excel('invoices_new.xlsx')

### In the excel document, please make sure to expand each of the columns, for viewing the table(s) properly
### Do not keep the excel file open while re running the code, which will prompt a permission error, so close the file before you rerun the code.